# Heat Equation with Adjoint Gradients

In this demo we are interested in time-dependent problems, namely the heat equation:

$$
\begin{align}
\frac{\partial u}{\partial t} - \Delta u &= 0 \quad \text{in } \Omega \times (0, T] \\
u(\mathbf{x}, 0) &= g(\mathbf{x}) \quad \text{in } \Omega
\end{align}
$$

where $\Omega$ is the unit square, $u$ is the temperature, and $g$ is the initial temperature. The problem has homogeneous Neumann boundary conditions.

For the initial temperature we choose:

$$
g(x,y) = \sin(2\pi x) \sin(2\pi y)
$$

To solve this problem we employ the Rothe method (method of lines), where the weak formulation is:

$$
\int_\Omega \frac{\partial u}{\partial t} v \, d\mathbf{x} + \int_\Omega \nabla u \cdot \nabla v \, d\mathbf{x} = 0
$$

for all $v \in H^1(\Omega)$. The time derivative is approximated by a backward Euler scheme:

$$
\int_\Omega \frac{u - u^n}{\Delta t} v \, d\mathbf{x} + \int_\Omega \nabla u \cdot \nabla v \, d\mathbf{x} = 0
$$

where $u^n$ is the solution at the previous time step.

The objective function is the $L^2$-norm of the temperature at the last time step:

$$
J(u) = \int_\Omega (u(T) - u_{\text{ref}})^2 \, d\mathbf{x} + \alpha \int_\Omega \nabla u_0 \cdot \nabla u_0 \, d\mathbf{x}
$$

where $u_{\text{ref}}$ is reference temperature data and $\alpha$ is a regularization parameter.

In [ ]:
import numpy as np
import ufl
from dolfinx import fem, mesh, nls
from mpi4py import MPI
from petsc4py.PETSc import ScalarType

from dolfinx_adjoint import *

Create the mesh and function space:

In [ ]:
domain = mesh.create_unit_square(MPI.COMM_WORLD, 32, 32, mesh.CellType.triangle)
V = fem.functionspace(domain, ("CG", 1))

Define time parameters:

In [ ]:
dt = 0.01
T = 0.05

Generate reference data by solving the heat equation with a known initial condition:

In [ ]:
# Set the initial values of the temperature variable u
true_initial = fem.Function(V, name="u_true_initial")
true_initial.interpolate(lambda x: np.sin(2 * np.pi * x[0]) * np.sin(2 * np.pi * x[1]))
u_prev = true_initial.copy()
u_next = true_initial.copy()

v = ufl.TestFunction(V)
dt_constant = fem.Constant(domain, ScalarType(dt))

# Set dirichlet boundary conditions
uD = fem.Function(V)
uD.interpolate(lambda x: 0.0 + 0.0 * x[0])
tdim = domain.topology.dim
fdim = tdim - 1
domain.topology.create_connectivity(fdim, tdim)

F = (
    ufl.inner((u_next - u_prev) / dt_constant, v) * ufl.dx
    + ufl.inner(ufl.grad(u_next), ufl.grad(v)) * ufl.dx
)
problem = fem.petsc.NonlinearProblem(F, u_next)
solver = nls.petsc.NewtonSolver(MPI.COMM_WORLD, problem)

t = 0.0
while t < T:
    solver.solve(u_next)
    u_prev.vector[:] = u_next.vector[:]
    t += dt
true_data = u_next.copy()

if MPI.COMM_WORLD.rank == 0:
    print("Reference data generated successfully!")

Create computational graph and solve with initial guess:

In [ ]:
graph_ = Graph()

# Set the initial values of the temperature variable u with an initial guess
initial_guess = fem.Function(V, name="initial_guess", graph=graph_)
initial_guess.interpolate(lambda x: 15.0 * x[0] * (1.0 - x[0]) * x[1] * (1.0 - x[1]))
u_prev = initial_guess.copy(graph=graph_, name="u_prev")
u_next = initial_guess.copy(graph=graph_, name="u_next")

F = (
    ufl.inner((u_next - u_prev) / dt_constant, v) * ufl.dx
    + ufl.inner(ufl.grad(u_next), ufl.grad(v)) * ufl.dx
)
t = 0.0
i = 0

# Store the states for the whole time-domain
u_iterations = [u_next.copy()]
while t < T:
    i += 1
    F = (
        ufl.inner((u_next - u_prev) / dt_constant, v) * ufl.dx
        + ufl.inner(ufl.grad(u_next), ufl.grad(v)) * ufl.dx
    )
    problem = fem.petsc.NonlinearProblem(F, u_next, graph=graph_)
    solver = nls.petsc.NewtonSolver(MPI.COMM_WORLD, problem, graph=graph_)

    solver.solve(u_next, graph=graph_, version=i)
    t += dt
    u_prev.assign(u_next, graph=graph_, version=i)

    # Store the iterations for visualization and testing
    u_iterations.append(u_next.copy())

if MPI.COMM_WORLD.rank == 0:
    print("Forward problem solved successfully!")

Define the objective function:

In [ ]:
alpha = fem.Constant(domain, ScalarType(1.0e-6))

J_form = (
    ufl.inner(true_data - u_next, true_data - u_next) * ufl.dx
    + alpha * ufl.inner(ufl.grad(initial_guess), ufl.grad(initial_guess)) * ufl.dx
)
J = fem.assemble_scalar(fem.form(J_form, graph=graph_), graph=graph_)

if MPI.COMM_WORLD.rank == 0:
    print(f"\nObjective function value: J(u) = {J}")

Compute the adjoint gradient with respect to the initial condition:

In [ ]:
dJdinit = graph_.backprop(id(J), id(initial_guess))

if MPI.COMM_WORLD.rank == 0:
    print("\n" + "="*60)
    print("Gradient Results:")
    print("="*60)
    print(f"J(u) = {J}")
    print(f"||dJ/du_0||_L2 = {np.sqrt(np.dot(dJdinit, dJdinit))}")
    print("="*60)